## Visualizar em um dashboard todas as instituições culturais do Brasil separadas por estados e região.

### Além de gerar a visão do mapa o instituto também precisa filtrar e visualizar informações sobre cada instituição como endereço, descrição, o tipo da instituição, região (norte, nordeste, centro-oeste, sul e sudeste).

In [0]:
# ✅ 1. Importações necessárias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg
import matplotlib.pyplot as plt


In [0]:
# ✅ 2. Inicialização da sessão Spark
spark = SparkSession.builder.getOrCreate()


In [0]:
# ✅ 3. Definição dos caminhos
silver_path = "/mnt/eoliveira/silver/museusbr"
gold_path = "/mnt/eoliveira/gold/museusbr"


In [0]:
# ✅ 4. Leitura da Silver
df_silver = spark.read.parquet(silver_path)




In [0]:
# ✅ 5. Confirmação do schema
print("✅ Schema da Silver:")
df_silver.printSchema()


In [0]:
# ✅ 6. Garantia que 'Regiao' existe
if 'Regiao' not in df_silver.columns:
    raise ValueError("❌ Coluna 'Regiao' não encontrada! Verifique o processamento da Silver.")


In [0]:
# ✅ 7. Criação da Gold: agregações e métricas
df_gold = (
    df_silver
    .groupBy("Regiao", "UF")
    .agg(
        count("*").alias("TotalInstituicoes"),
        avg("Latitude").alias("MediaLatitude"),
        avg("Longitude").alias("MediaLongitude")
    )
)


In [0]:
# ✅ 8. Exibir schema e amostra da Gold
print("✅ Schema da Gold:")
df_gold.printSchema()
df_gold.show(truncate=False)


In [0]:
# ✅ 9. Persistência da Gold
df_gold.write.mode("overwrite").parquet(gold_path)
print(f"✔️ Gold salva com sucesso em: {gold_path}")


In [0]:
# ✅ 10. Publicação: criação de view temporária
df_gold.createOrReplaceTempView("gold_view")
print("✅ View temporária 'gold_view' criada com sucesso.")


In [0]:
# ✅ 11. Total por região
resultado = spark.sql("""
    SELECT 
        Regiao,
        SUM(TotalInstituicoes) AS TotalPorRegiao
    FROM gold_view
    GROUP BY Regiao
    ORDER BY Regiao
""")
# ✅ 12. Exibir resultado
print("✅ Resultado da agregação por Região:")
resultado.show(truncate=False)


In [0]:
# Gráfico de barras: total por Região
totais_regiao = pandas_df.groupby('Regiao').size().reset_index(name='Total')

fig_bar = px.bar(
    totais_regiao,
    x='Regiao',
    y='Total',
    color='Regiao',
    text='Total',
    title='Total de Instituições por Região'
)

fig_bar.update_traces(textposition='outside')
fig_bar.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

fig_bar.show()


In [0]:
# ✅ 1. Agregação por Estado
resultado_uf = spark.sql("""
    SELECT 
        UF,
        SUM(TotalInstituicoes) AS TotalPorEstado
    FROM gold_view
    GROUP BY UF
    ORDER BY UF
""")

# ✅ 2. Conversão para Pandas
pandas_df_uf = resultado_uf.toPandas()

# ✅ 3. Plot
plt.figure(figsize=(14, 6))
bars = plt.bar(pandas_df_uf['UF'], pandas_df_uf['TotalPorEstado'], color='lightgreen')

plt.xlabel('Estado (UF)')
plt.ylabel('Total de Instituições')
plt.title('Total de Instituições Culturais por Estado')
plt.xticks(rotation=90)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# ✅ 4. Adicionar valores no topo de cada barra
for bar in bars:
    altura = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, altura, f'{int(altura)}', 
             ha='center', va='bottom', fontsize=9, color='black')

plt.tight_layout()
plt.show()


In [0]:
# Conversão da Gold para Pandas
pandas_df_gold = df_gold.toPandas()

# Conversão para Pandas com as colunas necessárias
pandas_df = df_silver.select(
    'Instituicoes', 'Endereco', 'Descricao', 'Tipo', 'Regiao', 'Latitude', 'Longitude'
).toPandas()




In [0]:
import folium

# Criar mapa centralizado
m = folium.Map(location=[-15.793889, -47.882778], zoom_start=4)

# Adicionar marcadores
for idx, row in pandas_df_gold.iterrows():
    popup_text = f"""
    <strong>{row['UF']} - {row['Regiao']}</strong><br>
    Total de Instituições: {row['TotalInstituicoes']}
    """
    folium.CircleMarker(
        location=[row['MediaLatitude'], row['MediaLongitude']],
        radius=5 + row['TotalInstituicoes'] * 0.05,  # tamanho proporcional
        popup=popup_text,
        color='blue',
        fill=True,
        fill_opacity=0.6
    ).add_to(m)

# Exibir mapa
m


In [0]:
import plotly.express as px

# Mapa interativo com Plotly
fig_map = px.scatter_mapbox(
    pandas_df,
    lat="Latitude",
    lon="Longitude",
    hover_name="Instituicoes",
    hover_data=["Endereco", "Descricao", "Tipo", "Regiao"],
    color="Regiao",
    zoom=3,
    height=600
)

fig_map.update_layout(mapbox_style="open-street-map")
fig_map.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig_map.show()


In [0]:
# ✅ Visualização da tabela completa
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

pandas_df[['Instituicoes', 'Endereco', 'Descricao', 'Tipo', 'Regiao']]
